In [1]:
from probe_experiment import *
from handcrafted_datasets import Dataset
from models import *
import torch
import torch.nn as nn
from sklearn.metrics import roc_curve, roc_auc_score

KeyboardInterrupt: 

In [ ]:
dataset = Dataset('ladders', 'C:\\Users\\andre\\go-ai\\data', 0.8, 0.2, 1024)

def get_train_loader():
    dataset.shuffle('train')
    return dataset.loader('train', max_ram_files=25)

def get_test_loader():
    return dataset.loader('test', max_ram_files=25)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
go_model = load_go_model_from_ckpt('model_ckpt.pth.tar', rm_prefix=True)
feat_models = [CutModel(go_model, i).cuda() for i in range(8)]
n_channels = [8, 64, 64, 64, 48, 48, 32, 32]
probe_models = [nn.Sequential(nn.Conv2d(nc, 2, 19, padding=9)).cuda() for nc in n_channels]

In [ ]:
for depth in range(0, 8):
    print('probing at depth %d' % depth)
    feat_model = feat_models[depth]
    probe_model = probe_models[depth]
    exp = ProbeExperiment(get_train_loader, get_test_loader, feat_model)
    config = {'name':'ladders/ladder_d%d'%depth, 'write_log':False, 'progress_bar':True, 'save_ckpt':False}
    optimizer = torch.optim.Adam(probe_model.parameters())
    exp.run(probe_model, criterion, optimizer, 5, config)

In [ ]:
aucs = []
aucs_break = []
for depth in range(8):
    feat_model = feat_models[depth]
    probe_model = probe_models[depth]
    exp = ProbeExperiment(get_train_loader, get_test_loader, feat_model)
    preds, labels = exp.get_predictions(probe_model, get_test_loader())
    
    auc = roc_auc_score(labels[:, 0].flatten(), preds[:, 0].flatten())
    
    auc_break = roc_auc_score(labels[:, 1].flatten(), preds[:, 1].flatten())
    
    aucs.append(auc)
    aucs_break.append(auc_break)
    print(auc, auc_break)

In [ ]:
feat_model = feat_models[0]
probe_model = probe_models[0]
exp = ProbeExperiment(get_train_loader, get_test_loader, feat_model)
preds, labels = exp.get_predictions(probe_model, get_test_loader())



In [ ]:
ladder_idxs = np.where(labels[:, 0].flatten() == 1)

In [ ]:
labels[:, 1].flatten()[ladder_idxs].mean()